## Configuration

In [1]:
from src.data_loading import get_merged_dataset, get_e_price_dataset, standardize_data, adjust_set_sizes

In [2]:
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact
import numpy as np

In [3]:
standardize_check = widgets.Checkbox(value=False, description='Standardize Data')
price_interval_ex_outliers_slider = widgets.IntRangeSlider(
    value=[-74.44, 222.01],  # Default range [min, max]
    min=-400,           # Minimum value
    max=900,         # Maximum value
    step=1,          # Step size
    description='Energy Price Interval',
    continuous_update=False  # Update only when sliding stops
)

# Create interactive widget
interact(adjust_set_sizes, 
         train=widgets.FloatSlider(min=0, max=0.9, step=0.01, value=0.8, description='Train Set Size'), 
         val=widgets.FloatSlider(min=0, max=0.2, step=0.01, value=0.05, description='Evaluation Set Size'),
         test=widgets.FloatSlider(min=0, max=0.2, step=0.01, value=0.15, description='Test Set Size'));

display(standardize_check, price_interval_ex_outliers_slider)


interactive(children=(FloatSlider(value=0.8, description='Train Set Size', max=0.9, step=0.01), FloatSlider(va…

Checkbox(value=False, description='Standardize Data')

IntRangeSlider(value=(-74, 222), continuous_update=False, description='Energy Price Interval', max=900, min=-4…

## Data loading

In [5]:
merged_df = get_merged_dataset()
e_price_df = get_e_price_dataset()

min_price, max_price = price_interval_ex_outliers_slider.value

merged_ex_out_df = merged_df.copy()[
    (merged_df['Price'] >= min_price) &
    (merged_df['Price'] <= max_price)
]

e_price_ex_out_df = merged_df.copy()[
    (merged_df['Price'] >= min_price) &
    (merged_df['Price'] <= max_price)
]

z_merged_df, z_actual_price_df, actual_price_scaler = standardize_data(merged_df.copy(), e_price_df.copy())
z_merged_ex_out_df, z_actual_ex_out_price_df, actual_price_ex_out_scaler = standardize_data(merged_ex_out_df.copy(), e_price_ex_out_df.copy())



## Feature Selection

## Prediction

### Prophet

### XGBoost

## Results